### Mathematical model calculation with symbolic equations 

In [ ]:
import sympy as sp
import numpy as np
import os
import pickle

from parameters import dict_with_params

t = sp.symbols("t", real=True)

######################################### basic state variable definitions
theta_v_y = sp.Function("theta_v_y")(t)
D_theta_v_y = sp.diff(theta_v_y, t)

psi_v_z = sp.Function("psi_v_z")(t)
D_psi_v_z = sp.diff(psi_v_z, t)

x_v = sp.Function("x_v")(t)
v_v_x = sp.diff(x_v, t)

######################################## basic parameters/expressions definitions
r, D, L = sp.symbols("r D L", positive=True)
J_v_x, J_v_y, J_v_z, m_v = sp.symbols("J_v_x J_v_y J_v_z m_v", positive=True)
J_w_xz, J_w_y, m_w = sp.symbols("J_w_xz J_w_y m_w", positive=True)
g = sp.symbols("g", positive=True)
T_theta, T_psi = sp.symbols("T_theta T_psi", cls=sp.Function)

T_L = (T_theta(t) - T_psi(t)) / 2
T_R = (T_theta(t) + T_psi(t)) / 2

v_cog_x = v_v_x + L * D_theta_v_y * sp.cos(theta_v_y)
v_cog_y = L * D_psi_v_z * sp.sin(theta_v_y)
v_cog_z = -L * D_theta_v_y * sp.sin(theta_v_y)

v_R_x = v_v_x + D / 2 * D_psi_v_z
v_L_x = v_v_x - D / 2 * D_psi_v_z

theta_wR_y = x_v / r + psi_v_z * D / 2 / r
theta_wL_y = x_v / r - psi_v_z * D / 2 / r

omega_R_y = v_R_x / r
omega_L_y = v_L_x / r

E_ws_kinetic_translational = 1 / 2 * m_w * v_R_x**2 + 1 / 2 * m_w * v_L_x**2

E_ws_kinetic_rotational = (
    1 / 2 * J_w_xz * D_psi_v_z**2
    + 1 / 2 * J_w_y * omega_R_y**2
    + 1 / 2 * J_w_xz * D_psi_v_z**2
    + 1 / 2 * J_w_y * omega_L_y**2
)

E_vehicle_kinetic_translational = (
    1 / 2 * m_v * v_cog_x**2 + 1 / 2 * m_v * v_cog_y**2 + 1 / 2 * m_v * v_cog_z**2
)

E_vehicle_kinetic_rotational = (
    1 / 2 * J_v_x * (-D_psi_v_z * sp.sin(theta_v_y)) ** 2
    + 1 / 2 * J_v_y * D_theta_v_y**2
    + 1 / 2 * J_v_z * (D_psi_v_z * sp.cos(theta_v_y)) ** 2
)

E_vehicle_potential = m_v * g * L * sp.cos(theta_v_y)

E_servomechanisms_potential = -T_R * (theta_wR_y - theta_v_y) - T_L * (
    theta_wL_y - theta_v_y
)

lagrangian = (
    E_ws_kinetic_translational
    + E_ws_kinetic_rotational
    + E_vehicle_kinetic_translational
    + E_vehicle_kinetic_rotational
    - E_vehicle_potential
    - E_servomechanisms_potential
)

lagrangian = sp.simplify(lagrangian)


In [ ]:

E1 = sp.Eq(sp.diff(sp.diff(lagrangian, v_v_x), t) - sp.diff(lagrangian, x_v), 0)
E2 = sp.Eq(
    sp.diff(sp.diff(lagrangian, D_theta_v_y), t) - sp.diff(lagrangian, theta_v_y), 0
)
E3 = sp.Eq(sp.diff(sp.diff(lagrangian, D_psi_v_z), t) - sp.diff(lagrangian, psi_v_z), 0)

D2_psi_v_z_t, D_psi_v_z_t, psi_v_z_t = sp.symbols(
    "D2_psi_v_z_t D_psi_v_z_t psi_v_z_t", real=True
)
D2_theta_v_y_t, D_theta_v_y_t, theta_v_y_t = sp.symbols(
    "D2_theta_v_y_t D_theta_v_y_t theta_v_y_t", real=True
)
D_v_v_x_t, v_v_x_t, x_v_t = sp.symbols("D_v_v_x_t v_v_x_t x_v_t", real=True)
T_theta_t, T_psi_t = sp.symbols("T_theta_t T_psi_t", real=True)

subs_dict = {
    x_v: x_v_t,
    v_v_x: v_v_x_t,
    sp.diff(v_v_x, t): D_v_v_x_t,
    theta_v_y: theta_v_y_t,
    sp.diff(theta_v_y, t): D_theta_v_y_t,
    sp.diff(theta_v_y, t, t): D2_theta_v_y_t,
    psi_v_z: psi_v_z_t,
    sp.diff(psi_v_z, t): D_psi_v_z_t,
    sp.diff(psi_v_z, t, t): D2_psi_v_z_t,
    T_theta(t): T_theta_t,
    T_psi(t): T_psi_t,
}

E1 = E1.subs(subs_dict)
E2 = E2.subs(subs_dict)
E3 = E3.subs(subs_dict)

E1 = sp.simplify(E1)
E2 = sp.simplify(E2)
E3 = sp.simplify(E3)

solution = sp.solve([E1, E2, E3], [D_v_v_x_t, D2_theta_v_y_t, D2_psi_v_z_t])

for key, value in solution.items():
    print(f"Key: {key}, Value: {value}")

D_v_v_x_f = solution[D_v_v_x_t]
D2_theta_v_y_f = solution[D2_theta_v_y_t]
D2_psi_v_z_f = solution[D2_psi_v_z_t]


In [ ]:

f = sp.Matrix(
    [v_v_x_t, D_v_v_x_f, D_theta_v_y_t, D2_theta_v_y_f, D_psi_v_z_t, D2_psi_v_z_f]
)
x = sp.Matrix([x_v_t, v_v_x_t, theta_v_y_t, D_theta_v_y_t, psi_v_z_t, D_psi_v_z_t])
u = sp.Matrix([T_theta_t, T_psi_t])
p = sp.Matrix([g, L, D, r, J_v_x, J_v_y, J_v_z, m_v, J_w_xz, J_w_y, m_w])

param_subs = {
    g: dict_with_params["g"],
    L: dict_with_params["L"],
    D: dict_with_params["D"],
    r: dict_with_params["r"],
    J_v_x: dict_with_params["J_v_x"],
    J_v_y: dict_with_params["J_v_y"],
    J_v_z: dict_with_params["J_v_z"],
    m_v: dict_with_params["m_v"],
    J_w_xz: dict_with_params["J_w_xz"],
    J_w_y: dict_with_params["J_w_y"],
    m_w: dict_with_params["m_w"],
}

f_subs_param = f.subs(param_subs)



folder_name = "pkl_objects"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)


file_name = "function_dxdt.pkl"
file = open(folder_name + "/" + file_name, "wb")
pickle.dump((f), file)
file.close()

# Calculating linear system as a Jacobian matrixes of non-linear system
D_f_D_x = f.jacobian(x)
D_f_D_u = f.jacobian(u)

A_s = D_f_D_x.subs(
    [
        (x_v_t, 0),
        (v_v_x_t, 0),
        (theta_v_y_t, 0),
        (D_theta_v_y_t, 0),
        (psi_v_z_t, 0),
        (D_psi_v_z_t, 0),
        (T_theta_t, 0),
        (T_psi_t, 0),
    ]
)
B_s = D_f_D_u.subs(
    [
        (x_v_t, 0),
        (v_v_x_t, 0),
        (theta_v_y_t, 0),
        (D_theta_v_y_t, 0),
        (psi_v_z_t, 0),
        (D_psi_v_z_t, 0),
        (T_theta_t, 0),
        (T_psi_t, 0),
    ]
)
A_s = sp.simplify(A_s)
B_s = sp.simplify(B_s)

A_n = A_s.subs(param_subs)
B_n = B_s.subs(param_subs)

A = np.array(A_n).astype(np.float64)
B = np.array(B_n).astype(np.float64)



print('Nonlinear model dx/dt=f(x,u,p) : ')
display(f)

print('After substitute of parameters : ')
display(f_subs_param)

print('Linear model matrixes A,B after linarization : ')
display(A_s)
display(B_s)

print('Their values after substitute of parameters : ')
display(A)
display(B)


